In [1]:
# Standard library
import os
import re
import sys
import warnings
from pathlib import Path

# Third-party - Data and scientific computing
import contextily as cx
import geopandas as gpd
import igraph as ig
import numpy as np
import pandas as pd
from pyproj import Geod
from tqdm import tqdm

# Shapely-specific imports for spatial analysis
import shapely
from shapely import STRtree
from shapely.geometry import LineString, MultiLineString, Point
from shapely.ops import nearest_points, snap

# Matplotlib-specific imports for figures
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, Normalize
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle
from matplotlib.ticker import FuncFormatter, MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Suppress warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [2]:
BASE_DIR = Path.cwd().parent
data_path = BASE_DIR / "input_files"
figure_path = BASE_DIR / "figures"
intermediate_results_path = BASE_DIR / 'intermediate_results'

In [3]:
gdf_hazards = gpd.read_parquet(intermediate_results_path / "main_network_hazard_exposure.parquet") 
base_network = gpd.read_parquet(intermediate_results_path / 'PERS_directed_final.parquet')

In [4]:
hospital_exposed_edges = gpd.read_parquet(intermediate_results_path / 'hospital_impacts.parquet').to_crs(gdf_hazards.crs)
factory_exposed_edges = gpd.read_parquet(intermediate_results_path / 'factory_impacts.parquet').to_crs(gdf_hazards.crs)
police_exposed_edges = gpd.read_parquet(intermediate_results_path / 'police_impacts.parquet').to_crs(gdf_hazards.crs)
fire_exposed_edges = gpd.read_parquet(intermediate_results_path / 'fire_impacts.parquet').to_crs(gdf_hazards.crs)
border_exposed_edges = gpd.read_parquet(intermediate_results_path / 'road_impacts.parquet').to_crs(gdf_hazards.crs)
port_exposed_edges = gpd.read_parquet(intermediate_results_path / 'port_impacts.parquet').to_crs(gdf_hazards.crs)
railway_exposed_edges = gpd.read_parquet(intermediate_results_path / 'rail_impacts.parquet').to_crs(gdf_hazards.crs)

FileNotFoundError: [WinError 2] Failed to open local file 'c:/Users/yma794/Documents/Serbia/Analysis - Copy2/intermediate_results/hospital_impacts.parquet'. Detail: [Windows error 2] The system cannot find the file specified.


In [ ]:
future_flood_change_rp =  gpd.read_parquet(intermediate_results_path / 'Future Floods change in RP.parquet').to_crs(gdf_hazards.crs)
future_rainfall_change =  gpd.read_parquet(intermediate_results_path / 'change in maximum daily precipitation rcp 85 period 2.paquet').to_crs(gdf_hazards.crs)

In [ ]:
def add_impact_column(base_gdf, edges_gdf, impact_col_name, predicate="intersects", agg="mean", col_to_focus = 'travel_time_impact'):
    """
    Spatially join edges to base_gdf, aggregate travel_time_impact per base index, and add as a new column.
    - predicate: 'intersects', 'within', 'contains', 'touches' (choose based on geometry semantics)
    - agg: 'mean', 'max', 'min', 'median' etc.
    """
    # Keep only needed columns to avoid bloat
    edges = edges_gdf[[col_to_focus, 'geometry']].copy()

    # Spatial join (left frame is base), this creates potential many-to-one matches
    joined = base_gdf.sjoin(edges, how='left', predicate=predicate)

    # Aggregate per left index (hazard_id)
    # Note: joined.index is the left index; the sjoin adds right index as 'index_right'
    agg_series = joined.groupby(joined.index)[col_to_focus].agg(agg)

    # Attach to base_gdf with a clear name
    base_gdf[impact_col_name] = agg_series.reindex(base_gdf.index)

    return base_gdf

# Add each impact column (choose your aggregator: 'mean' or 'max')
gdf_hazards = add_impact_column(gdf_hazards, hospital_exposed_edges, 'hospital_delay', predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, factory_exposed_edges,  'factory_delay',  predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, police_exposed_edges,   'police_delay',   predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, fire_exposed_edges,     'fire_delay',     predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, port_exposed_edges,  'port_delay',  predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, border_exposed_edges,   'border_delay',   predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, railway_exposed_edges,     'railway_delay',     predicate='intersects', agg='mean')
gdf_hazards = add_impact_column(gdf_hazards, future_flood_change_rp,   'future_flood_change',   predicate='intersects', agg='mean',col_to_focus='rp30_mean')
gdf_hazards = add_impact_column(gdf_hazards, future_rainfall_change,     'future_rainfall_change',     predicate='intersects', agg='mean',col_to_focus='max_rx1day_pct')

In [ ]:
# =============================================================================
# CLIMATE CRITICALITY METRIC - UPDATED METHODOLOGY
# =============================================================================

# Ensure gdf_hazards has a stable index
gdf_hazards = gdf_hazards.copy()
gdf_hazards.index.name = 'section_id'

# -----------------------------------------------------------------------------
# 1. PREPARE DATA - Rename columns to standard names
# -----------------------------------------------------------------------------

gdf_hazards = gdf_hazards.rename(columns={
    'max_depth': 'flood_depth',
    'dužina_sn': 'snow_drift',
    'datum_evid': 'landslide_date',
    # Add any other renames as needed
})

# Convert landslide_date to binary presence indicator
gdf_hazards['landslide_exposure'] = np.where(
    gdf_hazards['landslide_date'].astype(str).str.len() > 0, 1.0, 0.0
)

# -----------------------------------------------------------------------------
# 2. DEFINE METRIC GROUPS FOR EACH SUB-INDEX
# -----------------------------------------------------------------------------

# Hazard Exposure Sub-Index (H) - 5 metrics
hazard_metrics = [
    'flood_depth',              # F - Maximum inundation depth (cm)
    'future_rainfall_change',   # R - Projected % change in extreme rainfall
    'future_flood_change',      # C - Projected % change in river flood magnitude
    'landslide_exposure',       # L - Binary landslide exposure
    'snow_drift'                # S - Length affected by snow drift (km)
]

# National-Scale Travel Disruption Sub-Index (T) - 4 metrics
travel_metrics = [
    'phl',  # Passenger hours lost
    'thl',  # Tonnage hours lost
    'pkl',  # Passenger kilometers lost
    'tkl'   # Tonnage kilometers lost
]

# Local Accessibility Sub-Index (A) - 6 metrics
accessibility_metrics = [
    'hospital_delay',      # HOS - Hospital access delay
    'fire_delay',          # FIR - Fire station access delay
    'police_delay',        # POL - Police station access delay
    'port_delay',          # PRT - Port access delay (agriculture)
    'border_delay',        # BRD - Border crossing delay (agriculture/industry)
    'railway_delay'        # RWY - Railway station access delay (agriculture)
]

# All metrics combined
all_metrics = hazard_metrics + travel_metrics + accessibility_metrics

In [ ]:
# -----------------------------------------------------------------------------
# 3. ENSURE ALL COLUMNS EXIST AND HANDLE MISSING VALUES
# -----------------------------------------------------------------------------

for col in all_metrics:
    if col not in gdf_hazards.columns:
        print(f"Warning: Column '{col}' not found. Creating with zeros.")
        gdf_hazards[col] = 0.0
    # Convert to float and fill NaN with 0
    gdf_hazards[col] = gdf_hazards[col].astype(float).fillna(0.0)


In [ ]:
gdf_hazards

In [ ]:
# -----------------------------------------------------------------------------
# 4. MIN-MAX NORMALIZATION FUNCTION
# -----------------------------------------------------------------------------

def safe_minmax_normalize(series):
    """
    Normalize a series to [0, 1] using min-max normalization.
    Returns zeros if constant or all missing.
    """
    s = series.astype(float).fillna(0.0)
    min_val = s.min()
    max_val = s.max()
    
    if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
        return pd.Series(np.zeros(len(s)), index=s.index)
    
    return (s - min_val) / (max_val - min_val)

# -----------------------------------------------------------------------------
# 5. NORMALIZE ALL METRICS
# -----------------------------------------------------------------------------

print("Normalizing metrics...")

for col in all_metrics:
    norm_col = f'{col}_norm'
    gdf_hazards[norm_col] = safe_minmax_normalize(gdf_hazards[col])
    print(f"  {col}: min={gdf_hazards[col].min():.2f}, max={gdf_hazards[col].max():.2f}")

# -----------------------------------------------------------------------------
# 6. COMPUTE SUB-INDICES (Equal weights within each sub-index)
# -----------------------------------------------------------------------------

# Normalized column names
hazard_norm = [f'{col}_norm' for col in hazard_metrics]
travel_norm = [f'{col}_norm' for col in travel_metrics]
accessibility_norm = [f'{col}_norm' for col in accessibility_metrics]

# Hazard Exposure Sub-Index: H = (1/5) × (F + R + C + L + S)
gdf_hazards['H_hazard_exposure'] = gdf_hazards[hazard_norm].mean(axis=1)

# National-Scale Travel Disruption Sub-Index: T = (1/4) × (PHL + THL + PKL + TKL)
gdf_hazards['T_travel_disruption'] = gdf_hazards[travel_norm].mean(axis=1)

# Local Accessibility Sub-Index: A = (1/6) × (HOS + FIR + POL + PRT + BRD + RWY)
gdf_hazards['A_local_accessibility'] = gdf_hazards[accessibility_norm].mean(axis=1)

# -----------------------------------------------------------------------------
# 7. COMPUTE COMBINED CLIMATE CRITICALITY (Equal weights across sub-indices)
# -----------------------------------------------------------------------------

# CC = (1/3) × (H + T + A)
sub_indices = ['H_hazard_exposure', 'T_travel_disruption', 'A_local_accessibility']
gdf_hazards['CC_climate_criticality'] = gdf_hazards[sub_indices].sum(axis=1)

# -----------------------------------------------------------------------------
# 8. CLASSIFY INTO QUINTILES (5 categories based on 20% quantiles)
# -----------------------------------------------------------------------------

def classify_quintiles(series, labels=None):
    if labels is None:
        labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
    
    # Create an empty series to store results
    result = pd.Series(index=series.index, dtype='object')
    
    # 1. Identify zeros and assign "No criticality"
    is_zero = (series == 0)
    result[is_zero] = 'No criticality'
    
    # 2. Isolate non-zero values for quintile calculation
    non_zeros = series[~is_zero]
    
    if not non_zeros.empty:
        # Use qcut directly on non-zero values to ensure equal-sized bins
        # duplicates='drop' handles cases with many identical non-zero values
        quintiles = pd.qcut(non_zeros, 5, labels=labels, duplicates='drop')
        result[~is_zero] = quintiles
        
    return result

# Classify each sub-index
gdf_hazards['H_class'] = classify_quintiles(gdf_hazards['H_hazard_exposure'])
gdf_hazards['T_class'] = classify_quintiles(gdf_hazards['T_travel_disruption'])
gdf_hazards['A_class'] = classify_quintiles(gdf_hazards['A_local_accessibility'])

# Classify combined score
gdf_hazards['CC_class'] = classify_quintiles(gdf_hazards['CC_climate_criticality'])



In [ ]:
gdf_hazards.boxplot(['H_hazard_exposure','T_travel_disruption','A_local_accessibility','CC_climate_criticality'])

In [ ]:
gdf_hazards[['H_class','T_class','A_class']]

In [ ]:
# =============================================================================
# 2. Configuration: Colors, Labels, and Widths
# =============================================================================

labels = ['No criticality', 'Very Low', 'Low', 'Medium', 'High', 'Very High']
colors = ['#e0e0e0', '#edf8fb','#b3cde3','#8c96c6','#8856a7','#810f7c']
color_map = dict(zip(labels, colors))

width_mapping = {
    'No criticality': 0.2,
    'Very Low': 0.6,
    'Low': 0.9,
    'Medium': 1.3,
    'High': 2.0,
    'Very High': 3.0
}


# Ensure CRS is Web Mercator for contextily
gdf_hazards = gdf_hazards.to_crs(epsg=3857)
# If you have a separate base network for context:
# base_network = base_network.to_crs(epsg=3857)

# =============================================================================
# 4. Plotting Function and Figure Generation
# =============================================================================

def plot_panel(ax, gdf, class_col, letter, title):
    # Optional: Plot background network if you have one
    # base_network.plot(ax=ax, linewidth=0.1, color='lightgrey', alpha=0.5)
    
    # Plot each category in specific order (lowest to highest) 
    # so high criticality is on top
    for cat in labels:
        subset = gdf[gdf[class_col] == cat]
        if not subset.empty:
            subset.plot(
                ax=ax, 
                color=color_map[cat], 
                linewidth=width_mapping[cat],
                alpha=0.8 if cat != 'No criticality' else 0.4,
                zorder=labels.index(cat) # Higher criticality on top
            )
    
    # Add Basemap
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, attribution=False)
    
    # UI Elements
    ax.axis('off')
    ax.text(0.05, 0.95, letter, transform=ax.transAxes, fontsize=22, 
            fontweight='bold', verticalalignment='top',
            bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.9))
    ax.set_title(title, fontsize=14,fontweight='bold')

# Create the figure
fig, axes = plt.subplots(1, 3, figsize=(15, 7))

titles = ['Hazard-Exposure', 'National-Scale Travel Disruption', 'Local Accessibility']

# Loop through the axes and classification columns
for i, ax in enumerate(axes):
    plot_panel(axes[i], gdf_hazards, class_cols[i], chr(65+i), titles[i])

# =============================================================================
# 5. Shared Legend
# =============================================================================

legend_handles = [
    Patch(facecolor=color_map[lbl], label=lbl, edgecolor='grey', linewidth=0.5)
    for lbl in labels
]

fig.legend(
    handles=legend_handles,
    title='Criticality Level',
    loc='lower center',
    bbox_to_anchor=(0.5, 0.02),
    ncol=6,
    fontsize=13,
    title_fontsize=15,
    frameon=True
)

plt.tight_layout()
plt.subplots_adjust(bottom=0.18)

# Save or show
plt.savefig(figure_path / 'criticality_analysis_3panel.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CLIMATE CRITICALITY - SINGLE FIGURE (Mean)
# =============================================================================

# Calculate mean criticality
norm_cols = ['H_hazard_exposure', 'T_travel_disruption', 'A_local_accessibility']
gdf_hazards['criticality_mean'] = gdf_hazards[norm_cols].mean(axis=1)

# Classification function
def classify_quintiles(series, labels=None):
    if labels is None:
        labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
    result = pd.Series('No criticality', index=series.index)
    non_zero_mask = series > 0
    non_zeros = series[non_zero_mask]
    
    if not non_zeros.empty:
        bins = pd.qcut(non_zeros, 5, labels=labels, duplicates='drop')
        result[non_zero_mask] = bins.astype(str)
    return result

# Apply classification
gdf_hazards['mean_class'] = classify_quintiles(gdf_hazards['criticality_mean'])

# Ensure Web Mercator for plotting
gdf_hazards = gdf_hazards.to_crs(epsg=3857)

# Visualization configuration
labels = ['No criticality', 'Very Low', 'Low', 'Medium', 'High', 'Very High']
colors = ['#e0e0e0', '#ffffcc', '#a1dab4', '#41b6c4', '#2c7fb8', '#253494']
color_map = dict(zip(labels, colors))
width_mapping = {
    'No criticality': 0.2, 'Very Low': 0.6, 'Low': 1.0, 
    'Medium': 1.5, 'High': 2.2, 'Very High': 3.2
}

# Create single figure
fig, ax = plt.subplots(1, 1, figsize=(7, 9))

# Plot each category
for cat in labels:
    subset = gdf_hazards[gdf_hazards['mean_class'] == cat]
    if not subset.empty:
        subset.plot(
            ax=ax, 
            color=color_map[cat], 
            linewidth=width_mapping[cat],
            alpha=0.8 if cat != 'No criticality' else 0.4,
            zorder=labels.index(cat)
        )

# Add basemap and remove axes
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, attribution=False)
ax.axis('off')

# Legend
legend_handles = [
    Patch(facecolor=color_map[lbl], label=lbl, edgecolor='grey', linewidth=0.5)
    for lbl in labels
]

ax.legend(
    handles=legend_handles,
    title='Climate Criticality',
    loc='upper right',
    fontsize=14,
    title_fontsize=16,
    frameon=True,
    framealpha=0.9
)

plt.tight_layout()
plt.savefig(figure_path / 'climate_criticality_mean.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CLIMATE CRITICALITY - SUMMARY STATISTICS
# =============================================================================

print("="*80)
print("CLIMATE CRITICALITY SUMMARY STATISTICS")
print("="*80)

# Define classification columns
class_cols = ['H_class', 'T_class', 'A_class', 'mean_class']
class_names = ['Hazard Exposure', 'Travel Disruption', 'Local Accessibility', 'Combined Criticality']
index_cols = ['H_hazard_exposure', 'T_travel_disruption', 'A_local_accessibility', 'criticality_mean']

# =============================================================================
# 1. OVERALL DISTRIBUTION BY CRITICALITY CLASS
# =============================================================================

print("\n" + "="*80)
print("1. OVERALL DISTRIBUTION BY CRITICALITY CLASS")
print("="*80)

labels = ['No criticality', 'Very Low', 'Low', 'Medium', 'High', 'Very High']

for class_col, class_name in zip(class_cols, class_names):
    print(f"\n{class_name}:")
    print("-"*50)
    counts = gdf_hazards[class_col].value_counts()
    total = len(gdf_hazards)
    for label in labels:
        if label in counts.index:
            count = counts[label]
            pct = count / total * 100
            print(f"  {label:15s}: {count:5d} ({pct:5.1f}%)")
        else:
            print(f"  {label:15s}: {0:5d} ({0:5.1f}%)")

# =============================================================================
# 2. CRITICALITY BY ROAD CATEGORY
# =============================================================================

print("\n" + "="*80)
print("2. CRITICALITY BY ROAD CATEGORY")
print("="*80)

# Cross-tabulation: Road category vs Criticality class
for class_col, class_name in zip(class_cols, class_names):
    print(f"\n{class_name} by Road Category:")
    print("-"*60)
    
    # Create cross-tab with percentages
    ct = pd.crosstab(gdf_hazards['kategorija'], gdf_hazards[class_col], normalize='index') * 100
    
    # Reorder columns
    ct = ct.reindex(columns=[l for l in labels if l in ct.columns])
    
    print(ct.round(1).to_string())

# =============================================================================
# 3. MEAN CRITICALITY SCORES BY ROAD CATEGORY
# =============================================================================

print("\n" + "="*80)
print("3. MEAN CRITICALITY SCORES BY ROAD CATEGORY")
print("="*80)

road_summary = gdf_hazards.groupby('kategorija').agg({
    'H_hazard_exposure': ['count', 'mean', 'median', 'max'],
    'T_travel_disruption': ['mean', 'median', 'max'],
    'A_local_accessibility': ['mean', 'median', 'max'],
    'criticality_mean': ['mean', 'median', 'max']
}).round(4)

print(road_summary.to_string())

# =============================================================================
# 4. HIGH-TIER vs LOW-TIER ROADS COMPARISON
# =============================================================================

print("\n" + "="*80)
print("4. HIGH-TIER vs LOW-TIER ROADS COMPARISON")
print("="*80)

# Define road tiers
high_tier = ['IA', 'IM', 'IB']  # Motorways and main roads
low_tier = ['IIA', 'IIB']       # Regional roads

gdf_hazards['road_tier'] = np.where(
    gdf_hazards['kategorija'].isin(high_tier), 'High-Tier (IA/IM/IB)',
    np.where(gdf_hazards['kategorija'].isin(low_tier), 'Low-Tier (IIA/IIB)', 'Other')
)

# Compare mean scores
print("\nMean Criticality Scores by Road Tier:")
print("-"*60)
tier_summary = gdf_hazards.groupby('road_tier')[index_cols].mean().round(4)
print(tier_summary.to_string())

# Compare distribution of criticality classes
print("\n\nCombined Criticality Distribution by Road Tier (%):")
print("-"*60)
tier_ct = pd.crosstab(gdf_hazards['road_tier'], gdf_hazards['mean_class'], normalize='index') * 100
tier_ct = tier_ct.reindex(columns=[l for l in labels if l in tier_ct.columns])
print(tier_ct.round(1).to_string())

# Statistical test (if needed)
from scipy import stats
high_tier_scores = gdf_hazards[gdf_hazards['road_tier'] == 'High-Tier (IA/IM/IB)']['criticality_mean']
low_tier_scores = gdf_hazards[gdf_hazards['road_tier'] == 'Low-Tier (IIA/IIB)']['criticality_mean']

if len(high_tier_scores) > 0 and len(low_tier_scores) > 0:
    stat, pvalue = stats.mannwhitneyu(high_tier_scores, low_tier_scores, alternative='two-sided')
    print(f"\nMann-Whitney U test (High-Tier vs Low-Tier):")
    print(f"  U-statistic: {stat:.2f}")
    print(f"  p-value: {pvalue:.6f}")
    print(f"  Significant difference: {'Yes' if pvalue < 0.05 else 'No'}")

# =============================================================================
# 5. TOP 10 MOST CRITICAL SECTIONS (COMBINED)
# =============================================================================

print("\n" + "="*80)
print("5. TOP 10 MOST CRITICAL SECTIONS (Combined Criticality)")
print("="*80)

top_cols = ['oznaka_deo', 'kategorija', 'oznaka_put', 'naziv_poce', 'naziv_zavr',
            'H_hazard_exposure', 'T_travel_disruption', 'A_local_accessibility', 
            'criticality_mean', 'mean_class']
top_cols = [c for c in top_cols if c in gdf_hazards.columns]

top10_combined = gdf_hazards.nlargest(10, 'criticality_mean')[top_cols]
print(top10_combined.to_string())

# =============================================================================
# 6. TOP 5 PER SUB-INDEX
# =============================================================================

print("\n" + "="*80)
print("6. TOP 5 SECTIONS PER SUB-INDEX")
print("="*80)

for index_col, class_name in zip(index_cols[:3], class_names[:3]):
    print(f"\n{class_name} - Top 5:")
    print("-"*60)
    display_cols = ['oznaka_deo', 'kategorija', 'oznaka_put', 'naziv_poce', 'naziv_zavr', index_col]
    display_cols = [c for c in display_cols if c in gdf_hazards.columns]
    print(gdf_hazards.nlargest(5, index_col)[display_cols].to_string())

# =============================================================================
# 7. SHARE OF "HIGH" + "VERY HIGH" BY ROAD CATEGORY
# =============================================================================

print("\n" + "="*80)
print("7. SHARE OF HIGH/VERY HIGH CRITICALITY BY ROAD CATEGORY")
print("="*80)

high_crit_labels = ['High', 'Very High']

for class_col, class_name in zip(class_cols, class_names):
    print(f"\n{class_name}:")
    print("-"*50)
    
    # Calculate % of High + Very High per road category
    high_crit_pct = gdf_hazards.groupby('kategorija').apply(
        lambda x: (x[class_col].isin(high_crit_labels)).sum() / len(x) * 100
    ).sort_values(ascending=False)
    
    print("Road Category | % High/Very High | Count High/Very High | Total")
    for cat in high_crit_pct.index:
        pct = high_crit_pct[cat]
        count_high = gdf_hazards[(gdf_hazards['kategorija'] == cat) & 
                                  (gdf_hazards[class_col].isin(high_crit_labels))].shape[0]
        total = gdf_hazards[gdf_hazards['kategorija'] == cat].shape[0]
        print(f"  {cat:12s} | {pct:14.1f}% | {count_high:20d} | {total}")

# =============================================================================
# 8. CORRELATION BETWEEN SUB-INDICES
# =============================================================================

print("\n" + "="*80)
print("8. CORRELATION BETWEEN SUB-INDICES")
print("="*80)

corr_matrix = gdf_hazards[index_cols].corr()
print(corr_matrix.round(3).to_string())

# =============================================================================
# 9. SUMMARY TABLE FOR TEXT
# =============================================================================

print("\n" + "="*80)
print("9. SUMMARY TABLE FOR TEXT")
print("="*80)

summary_data = []
for cat in gdf_hazards['kategorija'].dropna().unique():
    subset = gdf_hazards[gdf_hazards['kategorija'] == cat]
    n_sections = len(subset)
    
    # % in High/Very High for each sub-index
    pct_H_high = (subset['H_class'].isin(high_crit_labels)).sum() / n_sections * 100
    pct_T_high = (subset['T_class'].isin(high_crit_labels)).sum() / n_sections * 100
    pct_A_high = (subset['A_class'].isin(high_crit_labels)).sum() / n_sections * 100
    pct_CC_high = (subset['mean_class'].isin(high_crit_labels)).sum() / n_sections * 100
    
    summary_data.append({
        'Road Category': cat,
        'N Sections': n_sections,
        '% High/VH Hazard': f"{pct_H_high:.1f}%",
        '% High/VH Travel': f"{pct_T_high:.1f}%",
        '% High/VH Access': f"{pct_A_high:.1f}%",
        '% High/VH Combined': f"{pct_CC_high:.1f}%"
    })

summary_df = pd.DataFrame(summary_data).sort_values('Road Category')
print(summary_df.to_string(index=False))

# =============================================================================
# 10. KEY FINDINGS FOR TEXT
# =============================================================================

print("\n" + "="*80)
print("10. KEY FINDINGS FOR TEXT")
print("="*80)

total_sections = len(gdf_hazards)
high_vh_combined = gdf_hazards['mean_class'].isin(high_crit_labels).sum()
pct_high_vh = high_vh_combined / total_sections * 100

print(f"""
KEY STATISTICS:
- Total road sections analyzed: {total_sections}
- Sections with High/Very High combined criticality: {high_vh_combined} ({pct_high_vh:.1f}%)

ROAD TIER COMPARISON:
- High-tier roads (IA/IM/IB) mean criticality: {high_tier_scores.mean():.4f}
- Low-tier roads (IIA/IIB) mean criticality: {low_tier_scores.mean():.4f}

TOP CRITICAL SECTION:
""")

top1 = gdf_hazards.nlargest(1, 'criticality_mean').iloc[0]
print(f"  Road: {top1.get('oznaka_put', 'N/A')} ({top1.get('kategorija', 'N/A')})")
print(f"  Section: {top1.get('naziv_poce', 'N/A')} → {top1.get('naziv_zavr', 'N/A')}")
print(f"  Combined Criticality Score: {top1['criticality_mean']:.4f}")
print(f"  H: {top1['H_hazard_exposure']:.4f}, T: {top1['T_travel_disruption']:.4f}, A: {top1['A_local_accessibility']:.4f}")

In [ ]:
gdf_hazards.to_excel(intermediate_results_path / 'VUA_Climate_Criticality_PERS.xlsx')

In [ ]:
gdf_hazards.sort_values('criticality_sum',ascending=False)

In [ ]:
# 1) Prepare data: drop zero or missing criticality
gdf_plot = gdf_hazards.copy().to_crs(3857)
gdf_plot = gdf_plot[gdf_plot['criticality_sum'].fillna(0) > 0.1]

# 2) Define bins and labels
bins = [0, 1, 2, 3, np.inf]
labels = ['0–1', '1–2', '2–3', '>3']

# 3) Assign classes
gdf_plot['criticality_class'] = pd.cut(
    gdf_plot['criticality_sum'],
    bins=bins,
    labels=labels,
    include_lowest=False,  # (0–1], (1–2], (2–3], (3, inf)
    right=True
)

# 4) Colors (low to high) — swap to your preferred palette if needed
colors = ['#fcae91','#fb6a4a','#de2d26','#a50f15']
color_map = dict(zip(labels, colors))

# 5) Figure and axis
fig, ax = plt.subplots(1, 1, figsize=(10, 9), facecolor='white')

# Base network (light grey) — plotted above basemap, below criticality
base_network.to_crs(3857).plot(ax=ax, linewidth=0.15, color='lightgrey', alpha=0.6, zorder=2)

# Plot each class
for cls in labels:
    sub = gdf_plot[gdf_plot['criticality_class'] == cls]
    if len(sub) == 0:
        continue
    sub.plot(
        ax=ax,
        color=color_map[cls],
        linewidth=1.5,
        alpha=0.95,
        zorder=4
    )

# Legend
legend_handles = [
    Patch(facecolor=color_map[lbl], edgecolor='black', linewidth=0.3, label=lbl)
    for lbl in labels
]
ax.legend(
    handles=legend_handles,
    title='Criticality (sum)',
    loc='upper right',
    frameon=True,
    fancybox=True,
    framealpha=0.9,
    fontsize=12,
    title_fontsize=14
)

cx.add_basemap(
    ax=ax,
    source=cx.providers.CartoDB.Positron,
    alpha=0.8,       # light background
    attribution=False,
    zorder=1)


# Cosmetics
ax.set_aspect('equal')
ax.axis('off')


plt.savefig(figure_path / 'criticality_sum_map.png', dpi=300, bbox_inches='tight')


In [ ]:
# --- Settings ---
TOP_METHOD = "quantile"    # "quantile" or "topN" or "cutoff"
TOP_Q = 0.90               # top 10% by criticality_sum
TOP_N = 1000               # if TOP_METHOD == "topN"
CUTOFF_VAL = 3.0           # if TOP_METHOD == "cutoff" (e.g., >3 in your legend)

# --- Prep ---
gdf = gdf_hazards.copy()

gdf["len_km"] = gdf['road_length']

# Identify normalized submetrics used in the sum
norm_cols = sorted([c for c in gdf.columns if c.endswith("_norm")])
if "criticality_sum" not in gdf.columns:
    # If not present, create it as sum of all _norm columns
    gdf["criticality_sum"] = gdf[norm_cols].sum(axis=1)

# --- Select top critical segments ---
if TOP_METHOD == "quantile":
    thr = gdf["criticality_sum"].quantile(TOP_Q)
    top_mask = gdf["criticality_sum"] >= thr
elif TOP_METHOD == "topN":
    top_idx = gdf["criticality_sum"].nlargest(TOP_N).index
    top_mask = gdf.index.isin(top_idx)
elif TOP_METHOD == "cutoff":
    top_mask = gdf["criticality_sum"] > CUTOFF_VAL
else:
    raise ValueError("TOP_METHOD must be one of: quantile, topN, cutoff")

top = gdf.loc[top_mask].copy()
rest = gdf.loc[~top_mask].copy()

print(f"Total segments: {len(gdf):,}")
print(f"Top subset size: {len(top):,} ({len(top)/len(gdf)*100:.1f}%)")
print(f"Top subset total length: {top['len_km'].sum():,.1f} km")
print(f"Mean criticality_sum (top): {top['criticality_sum'].mean():.2f}")
print(f"Mean criticality_sum (rest): {rest['criticality_sum'].mean():.2f}")

# --- 1) Which road categories dominate in the top subset? ---
cat_summary_top = (top.groupby("kategorija")
                      .agg(n_segments=("kategorija", "size"),
                           length_km=("len_km", "sum"),
                           avg_crit=("criticality_sum", "mean"))
                      .sort_values(["n_segments", "length_km"], ascending=False))
cat_summary_all = (gdf.groupby("kategorija")
                      .agg(n_segments=("kategorija", "size"),
                           length_km=("len_km", "sum"))
                      .sort_values(["n_segments", "length_km"], ascending=False))

# Add shares vs. network totals
cat_summary_top["share_count_%"] = (cat_summary_top["n_segments"] / len(top) * 100).round(1)
cat_summary_top["share_length_%"] = (cat_summary_top["length_km"] / top["len_km"].sum() * 100).round(1)

print("\nTop critical subset by road category (count, length, and avg criticality):")
print(cat_summary_top.round(2).to_string())

print("\nAll segments by road category (for baseline context):")
print(cat_summary_all.round(2).to_string())

# --- 2) Which submetrics contributed most to top rankings (overall)? ---
# Contribution here is the sum of normalized values per column in the top subset.
submetric_contrib_top = top[norm_cols].sum().sort_values(ascending=False)
submetric_contrib_top_pct = (submetric_contrib_top / submetric_contrib_top.sum() * 100).round(1)
submetric_contrib_df = pd.DataFrame({
    "total_contribution": submetric_contrib_top.round(2),
    "share_%": submetric_contrib_top_pct
}).sort_values("total_contribution", ascending=False)

print("\nSubmetric contributions in top subset (overall):")
print(submetric_contrib_df.to_string())

# --- 3) Which submetric dominates within each road category (top subset)? ---
# For each category, compute contribution shares across submetrics
by_cat_contrib = (top.groupby("kategorija")[norm_cols].sum())
by_cat_contrib_share = by_cat_contrib.div(by_cat_contrib.sum(axis=1), axis=0) * 100.0
# Identify dominant submetric per category
dominant_metric_per_cat = by_cat_contrib_share.idxmax(axis=1).to_frame("dominant_submetric")
dominant_share_per_cat = by_cat_contrib_share.max(axis=1).round(1).to_frame("dominant_share_%")
dominant_cat_df = (pd.concat([dominant_metric_per_cat, dominant_share_per_cat], axis=1)
                     .sort_values("dominant_share_%", ascending=False))

print("\nDominant submetric per road category in the top subset:")
print(dominant_cat_df.to_string())

# --- 4) Quick list of top segments (IDs) with their category and contributions ---
# Show top 20 by criticality_sum with breakdown across submetrics
top_breakdown = top.loc[top["criticality_sum"].nlargest(20).index,
                        ["kategorija", "criticality_sum"] + norm_cols].round(3)
print("\nTop 20 segments by criticality_sum with submetric breakdown:")
print(top_breakdown.to_string())

# --- 5) Optional: overall shares by band used in your map legend ---
bands = pd.cut(gdf["criticality_sum"], bins=[0,1,2,3,np.inf], labels=["0–1","1–2","2–3",">3"], right=True)
band_share = bands.value_counts(normalize=True).sort_index() * 100
print("\nNetwork share by criticality_sum bands (0–1, 1–2, 2–3, >3):")
print(band_share.round(1).to_string())

# --- 6) Optional: per-category median criticality and length in the top subset ---
cat_medians = top.groupby("kategorija").agg(
    median_crit=("criticality_sum", "median"),
    length_km=("len_km", "sum")
).sort_values("median_crit", ascending=False)
print("\nPer-category median criticality and total length (top subset):")
print(cat_medians.round(2).to_string())


In [ ]:
# --- Vehicle Hours Lost (VHL) fun facts for the top subset ---

def _find_col(df, candidates):
    """Return the first column found in df among candidates, case-insensitive."""
    cols_lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

# 1) Identify or construct raw VHL
# Try existing raw VHL first
vhl_col = _find_col(gdf, ["VHL", "vhl", "vehicle_hours_lost", "vehicle_hours_lost_daily", "vhl_daily"])

# Else compute from delay and AADT if available
delay_col = _find_col(gdf, ["avg_delay_min", "avg_time_disruption_min", "delay_min", "avg_delay_minutes"])
aadt_col  = _find_col(gdf, ["AADT", "aadt"])

if vhl_col is None and (delay_col is not None and aadt_col is not None):
    vhl_col = "VHL_computed_daily"
    gdf[vhl_col] = (gdf[delay_col].astype(float) / 60.0) * gdf[aadt_col].astype(float)

# Fallback to normalized proxy only if nothing else is available
using_normalized_proxy = False
if vhl_col is None:
    vhl_norm_col = _find_col(gdf, ["vhl_norm"])
    if vhl_norm_col is None:
        raise ValueError("Could not find raw VHL nor ingredients to compute it, and vhl_norm is also missing.")
    vhl_col = vhl_norm_col
    using_normalized_proxy = True

# 2) Prepare series for top and rest
vhl_top = top[vhl_col].astype(float)
vhl_rest = rest[vhl_col].astype(float)
vhl_all = gdf[vhl_col].astype(float)

# 3) Core fun facts
def pct(x): 
    return f"{100.0 * x:.1f}%"

if not using_normalized_proxy:
    # Thresholds in daily vehicle-hours
    thresholds = [1000, 5000, 10000]

    share_total_vhl_top = (vhl_top.sum() / vhl_all.sum()) if vhl_all.sum() > 0 else np.nan
    med_top = np.nanmedian(vhl_top)
    p90_top = np.nanpercentile(vhl_top, 90)
    p95_top = np.nanpercentile(vhl_top, 95)
    max_top = np.nanmax(vhl_top)

    print("\n--- Vehicle hours lost: ready-to-paste statements ---")
    print(f"The top critical subset accounts for {pct(share_total_vhl_top)} of total daily vehicle hours lost on the network.")
    print(f"Within the top subset the median daily vehicle hours lost per segment is {med_top:,.0f}, with the 90th and 95th percentiles at {p90_top:,.0f} and {p95_top:,.0f}, and a maximum of {max_top:,.0f}.")

    # Counts above thresholds in top vs rest
    for thr in thresholds:
        n_top = int((vhl_top >= thr).sum())
        n_rest = int((vhl_rest >= thr).sum())
        print(f"{n_top} segments in the top subset exceed {thr:,.0f} daily vehicle hours lost "
              f"(compared with {n_rest} in the rest of the network).")

    # Which road classes most often exceed thresholds
    top_with_vhl = top.assign(_vhl=vhl_top.values).copy()
    for thr in thresholds:
        exceed = (top_with_vhl[top_with_vhl["_vhl"] >= thr]
                  .groupby("kategorija")
                  .size()
                  .sort_values(ascending=False))
        if exceed.empty:
            continue
        print(f"\nAmong top segments exceeding {thr:,.0f} daily vehicle hours lost, the most frequent categories are:")
        for cat, cnt in exceed.items():
            print(f"  {cat}: {cnt} segments")

else:
    # Normalized proxy mode: use quantiles rather than absolute thresholds
    share_total_vhl_top = (vhl_top.sum() / vhl_all.sum()) if vhl_all.sum() > 0 else np.nan
    med_top = np.nanmedian(vhl_top)
    p90_top = np.nanpercentile(vhl_top, 90)
    p95_top = np.nanpercentile(vhl_top, 95)
    max_top = np.nanmax(vhl_top)

    print("\n--- Vehicle hours lost (normalized proxy): ready-to-paste statements ---")
    print(f"The top critical subset accounts for {pct(share_total_vhl_top)} of the summed normalized vehicle hours lost metric.")
    print(f"Within the top subset the median normalized value is {med_top:,.3f}, with the 90th and 95th percentiles at {p90_top:,.3f} and {p95_top:,.3f}, and a maximum of {max_top:,.3f}.")

    # Report which categories dominate the top decile of the proxy
    q_thr = np.nanpercentile(vhl_all, 90)
    exceed = (top[vhl_col] >= q_thr)
    by_cat = top.loc[exceed].groupby("kategorija").size().sort_values(ascending=False)
    if not by_cat.empty:
        print("\nWithin the top subset, categories most represented in the top decile of the normalized vehicle hours lost proxy are:")
        for cat, cnt in by_cat.items():
            print(f"  {cat}: {cnt} segments")

# 4) Optional: Top segments by VHL for quick cross-checking
n_show = 20
cols_show = ["kategorija", "criticality_sum"]
if vhl_col not in cols_show:
    cols_show = ["kategorija", "criticality_sum", vhl_col]
top_by_vhl = top.sort_values(by=vhl_col, ascending=False).head(n_show)[cols_show]
print(f"\nTop {n_show} segments by {vhl_col}:")
print(top_by_vhl.to_string(index=True))
